In [1]:
import time
import blot
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from dateutil import parser

In [7]:
data = pd.read_csv("../data/storage.csv")
data["time"] = pd.to_datetime(data["time"], unit="s")
# data["time"] =  data["time"].apply(lambda ts: datetime.fromtimestamp(ts))
data = data.set_index(["time"])
data["value"] = data["value"] / 1000
data = data.iloc[(data.index > datetime(2020, 8, 30)) & (data.index < datetime(2020, 9, 30))]
data.blot(theme="white")

In [3]:
from scipy.fftpack import fft, fftfreq

fft_series = fft(data["value"].values)
power = np.abs(fft_series)
sample_freq = fftfreq(fft_series.size)

pos_mask = np.where(sample_freq > 0)
freqs = sample_freq[pos_mask]
powers = power[pos_mask]

top_k_seasons = 3
# top K=3 index
top_k_idxs = np.argpartition(powers, -top_k_seasons)[-top_k_seasons:]
top_k_power = powers[top_k_idxs]
fft_periods = (1 / freqs[top_k_idxs]).astype(int)

print(f"top_k_power: {top_k_power}")
print(f"fft_periods: {fft_periods}")

top_k_power: [ 614.8105282   890.33273899 1831.167168  ]
fft_periods: [ 72 278 292]


In [4]:
pd.Series(np.sort(powers)[::-1][:10]).rename("power").blot(kind="scatter", symbol="diamond", size=6)

In [5]:

from statsmodels.tsa.stattools import acf

# Expected time period
for lag in fft_periods:
    # lag = fft_periods[np.abs(fft_periods - time_lag).argmin()]
    acf_score = acf(data["value"].values, nlags=lag)[-1]
    print(f"lag: {lag} fft acf: {acf_score}")


expected_lags = np.array([timedelta(hours=12)/timedelta(minutes=5), timedelta(days=1)/timedelta(minutes=5), timedelta(days=7)/timedelta(minutes=5)]).astype(int)
for lag in expected_lags:
    acf_score = acf(data["value"].values, nlags=lag, fft=False)[-1]
    print(f"lag: {lag} expected acf: {acf_score}")


lag: 72 fft acf: 0.07405431832776994
lag: 278 fft acf: 0.7834457453491087
lag: 292 fft acf: 0.8259822269757922
lag: 144 expected acf: -0.5942986094704665
lag: 288 expected acf: 0.8410792774898174
lag: 2016 expected acf: 0.5936030431473589


/Users/shenmengjia/opt/anaconda3/envs/shooter/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:667: FutureWarning:

fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.

